# M2608.001300 Machine Learning<br> Assignment #3 Training Convolutional Neural Networks (Pytorch)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jongwan Kim, May 2021

**For understanding of this work, please carefully look at given PPT file.**

Now, you're going to leave behind your implementations and instead migrate to one of popular deep learning frameworks, **PyTorch**. <br>
In this notebook, you will learn how to train convolutional neural networks (CNNs) for classifying images in the CIFAR-10 dataset. <br>
There are **2 sections**, and in each section, you need to follow the instructions to complete the skeleton codes and explain them.

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.


### Some helpful tutorials and references for assignment #3:
- [1] Pytorch official documentation. [[link]](https://pytorch.org/docs/stable/index.html)
- [2] Stanford CS231n lectures. [[link]](http://cs231n.stanford.edu/)
- [3] Szegedy et al., "Going deeper with convolutions", CVPR 2015. [[pdf]](http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf)

## 1. Load datasets
The CIFAR-10 dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    print(np.transpose(npimg, (1, 2, 0)).shape)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

(240, 274, 3)


ship horse  bird   cat
torch.Size([3, 32, 32])


In [4]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
# print size of single image
print(images[1].shape)

## 2. Training a small CNN model

#### CNN architecture in order:
* 7x7 Convolutional layer with 8 filters, strides of 1, and ReLU activation
* 2x2 Max pooling layer with strides of 2
* 4x4 Convolutional layer with 16 filters, strides of 1, and ReLU activation
* 2x2 Max pooling layer with strides of 2
* Fully connected layer with 100 output units and ReLU activation
* Fully connected layer with 80 output units and ReLU activation
* Fully connected layer with 10 output units and linear activation
* You can use any padding option.

#### Training setup:
* Loss function: Softmax cross entropy
* Optimizer: Gradient descent with 0.001 learning rate
* Batch size: 4 (Things you can try to change)
* Training epoch: 2

In [5]:
# Define a CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
        self.Layer_1 = nn.Sequential(
            nn.Conv2d(3, 8,  7),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 4),
            nn.ReLU(True),
            nn.MaxPool2d(2),
        )
        
        self.FC_Layer= nn.Sequential(
            nn.Linear(400,100),
            nn.ReLU(True),
            nn.Linear(100,50),
            nn.ReLU(True),
            nn.Linear(50,10),
        )

        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################

    def forward(self, x):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
        x = self.Layer_1(x)
        x = x.view(-1, 400)
        x = self.FC_Layer(x)
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################
        return x

net = Net()

In [6]:
# Training on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [7]:
# Define a Loss function and optimizer
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################

In [8]:
# Function to train the network

def train(net, trainloader, max_epoch, crit, opt, model_path='./cifar_net.pth'):

    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
        
            # Training on GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = crit(outputs, labels)
            loss.backward()
            opt.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')
    torch.save(net.state_dict(), model_path)
    print('Saved Trained Model')

In [9]:
PATH = './cifar_net.pth'
epoch = 2
train(net, trainloader, epoch, criterion, optimizer, PATH)

[1,   100] loss: 2.305
[1,   200] loss: 2.304
[1,   300] loss: 2.303
[1,   400] loss: 2.300
[1,   500] loss: 2.298
[1,   600] loss: 2.295
[1,   700] loss: 2.292
[1,   800] loss: 2.286
[1,   900] loss: 2.275
[1,  1000] loss: 2.256
[2,   100] loss: 2.224
[2,   200] loss: 2.149
[2,   300] loss: 2.102
[2,   400] loss: 2.052
[2,   500] loss: 1.982
[2,   600] loss: 1.954
[2,   700] loss: 1.953
[2,   800] loss: 1.925
[2,   900] loss: 1.906
[2,  1000] loss: 1.897
Finished Training
Saved Trained Model


In [10]:
# function to calculate accuracy
def print_accuracy(net, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            # Inference on GPU
            images = images.to(device)
            labels = labels.to(device)
        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the %d test images: %d %%' % (total,
        100 * correct / total))

In [11]:
# load trained model then test
net.load_state_dict(torch.load(PATH))
print_accuracy(net, testloader)

Accuracy of the network on the 10000 test images: 33 %


## 3. Design a better model on CIFAR-10
Now it's your job to experiment with CNNs to train a model that achieves **<font color=red>>= 70% accuracy on the test set</font>** of CIFAR-10. <br> You can use the implemented *inception class* below.

### Things you can try to change:
- Batch size (input parameter of dataloader)
- Filter size
- Number of filters
- Pooling vs Strided Convolution
- Network architectures
- Optimizers
- Activation functions
- Regularizations
- Model ensembles

In [12]:
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)

In [13]:
# Define a CNN model
class BetterNet(nn.Module):
    def __init__(self):
        super(BetterNet, self).__init__()
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
        self.Layer_1 = nn.Sequential(
            nn.Conv2d(3, 64,  7, stride = 1, padding= 3),
            #nn.ReLU(True),
            nn.MaxPool2d(3,1,1),
            nn.Conv2d(64,  192, 1, 1, 1),
            #nn.ReLU(True),
            nn.MaxPool2d(3,1,1),
        )
        
        self.ILayer_1 = nn.Sequential(
            Inception(192, 64, 96, 128, 16, 32, 32),
            Inception(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(3,2,1),
        )

        self.ILayer_2 = nn.Sequential(
            Inception(480, 192, 96, 208, 16, 48, 64),
            Inception(512, 160, 112, 224, 24, 64, 64),
            Inception(512, 128, 128, 256, 24, 64, 64),
            Inception(512, 112, 144, 288, 32, 64, 64),
            Inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3,2,1),
        )

        self.ILayer_3 = nn.Sequential(
            Inception(832, 256, 160, 320, 32, 128, 128),
            Inception(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(9,1),
        )

        self.FC_Layer= nn.Sequential(
            nn.Linear(1024,100),
            nn.ReLU(True),
            nn.Linear(100,10),
            
        )
        """
        self.ILayer_2 = nn.Sequential(
            Inception(480, 160, 240, 224, 128, 64, 64),
            Inception(512, 160, 240, 240, 50, 62, 50),
            Inception(500, 240, 250, 240, 8, 16, 16),
            Inception(512, 200, 200, 100, 25, 50 ,50),
            Inception(400, 200, 300, 250, 100, 150, 100),
            nn.MaxPool2d(3,2,1),
        )

        self.ILayer_3 = nn.Sequential(
            Inception(700, 256, 600, 512, 64, 128, 128),
            Inception(1024, 384, 768, 384, 64, 128, 128),
            nn.AvgPool2d(9,1),
        )
        
        self.ILayer_2 = nn.Sequential(
            Inception(480, 160, 112, 224, 32, 64, 64),
            Inception(512, 160, 120, 240, 25, 56, 56),
            Inception(512, 240, 200, 240, 8, 16, 16),
            Inception(512, 200, 50, 100, 25, 50 ,50),
            Inception(528, 214, 128, 264, 50, 125, 125),
            nn.MaxPool2d(3,2,1),
        )
        
        self.ILayer_3 = nn.Sequential(
            Inception(832, 256, 256, 512, 64, 128, 128),
            Inception(1024, 384, 192, 384, 64, 128, 128),
            nn.AvgPool2d(9,1),
        )

        self.drop = nn.Dropout2d(0.5)
        self.FC_Layer= nn.Sequential(
            nn.Linear(1024,100),
            nn.ReLU(True),
            nn.Linear(100,10),
            
        )
        """
        
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################

    def forward(self, x):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
        
        out = self.Layer_1(x)
        
        out = self.ILayer_1(out)
        
        out = self.ILayer_2(out)
        
        out = self.ILayer_3(out)
        
        #out = self.drop(out)
        # we don't need drop out for only 2 epoch #
        
        out = out.view(-1,1024)
        
        out = self.FC_Layer(out)
        
        
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################
        return out

betternet = BetterNet()
betternet = betternet.to(device)

In [14]:
# Define a Loss function and optimizer
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(betternet.parameters(), lr=0.005, momentum= 0.9,nesterov = True, weight_decay = 0.001)
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################

In [15]:
PATH = './better_net.pth'
# Train
train(betternet, trainloader, 2, criterion, optimizer, PATH)
# Test
betternet.load_state_dict(torch.load(PATH))
print_accuracy(betternet, testloader)

[1,   100] loss: 1.973
[1,   200] loss: 1.649
[1,   300] loss: 1.469
[1,   400] loss: 1.385
[1,   500] loss: 1.299
[1,   600] loss: 1.247
[1,   700] loss: 1.184
[1,   800] loss: 1.138
[1,   900] loss: 1.106
[1,  1000] loss: 1.055
[2,   100] loss: 0.983
[2,   200] loss: 1.000
[2,   300] loss: 0.943
[2,   400] loss: 0.928
[2,   500] loss: 0.881
[2,   600] loss: 0.881
[2,   700] loss: 0.863
[2,   800] loss: 0.871
[2,   900] loss: 0.807
[2,  1000] loss: 0.831
Finished Training
Saved Trained Model
Accuracy of the network on the 10000 test images: 72 %


### Describe what you did here
In this cell you should also write an explanation of what you did, any additional features that you implemented, and any visualizations or graphs that you make in the process of training and evaluating your network.

_Tell us here_